In [1]:
#きれいなテキストを分配する
#TODO: code重複

In [1]:
from src.cleaner.auto_cleaner import clean_text
import argparse
import json
import os
from src.classify.Text2Vec import Text2Vec, texts2classes
from src.cleaner.auto_cleaner import clean_text, ml_clean_text
from gensim.models.fasttext import load_facebook_model
import joblib

In [9]:
streaming = True
base_dir = "../data/categorized"
length_threshold = 30  # 短い記事は捨てる
check_length = 200  # はじめのlengthだけで分類する


# load models
t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))
kmeans = joblib.load("../data/model/kmeans.pkl")


In [19]:

def make_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)

def proc(docs,database_name):
    # docsを処理する関数
    # ここに処理のロジックを実装します
    print(f"Processing {len(docs)} documents...")
    categories = texts2classes(docs, t2v, kmeans, length=check_length)

    for text, category in zip(docs, categories):
        #print(categories,text)
        save_dir = f"{base_dir}/{category}"

        make_dir(save_dir)
        data = json.dumps(
            {
                # "db": database_name, #特に必要ない｡storage 節約
                "text": text},
            ensure_ascii=False
        )
        database_name = database_name.replace(".json", "").replace(".gz", "")
        with open(f"{save_dir}/{database_name}.jsonl", "a") as f:
            f.write(data+"\n")

    return len(docs)


In [22]:
import sys
sys.path.append("../20integrate_texts/")
from loaders.loaders import *
length_threshold = 100
batch_size=10000

loader_dict={
    "NHK_School":NHKSchool_loader(),
    "WikiQA":wiki_qa_loader(),
    "Wiki":cleaned_wiki_loader(),
    "NHK_News":nhk_news_loader(),
    "aozora":aozora_bunko_loader(),
    "j_ronbun":j_research_loader(),
    #"cosmo":cosmo_loader(),
}

/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.
Filter: 100%|██████████| 1395760/1395760 [00:31<00:00, 44958.24 examples/s]


In [23]:



for doc_name,loader in loader_dict.items():
    docs = []
    lines = []
    for record in iter(loader):
        lines.append(record["text"])

    cnt=0
    for text in lines:
        text = clean_text(text)
        if len(text) < length_threshold:
            continue

        docs.append(text)
        if len(docs) == batch_size:
            # docsのコピーを作成してprocに渡す
            proc(docs[:],doc_name+str(cnt))
            # docsをリセット
            docs = []
            cnt+=1

    proc(docs[:],doc_name+str(cnt))

Processing 4522 documents...
